# Instalación de librerias

In [1]:
!pip install tensorflow numpy keras matplotlib scikit-learn seaborn pandas flask flask-cors


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pickle
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
import json

# Carga del modelo

In [3]:
!dir

 El volumen de la unidad C no tiene etiqueta.
 El n�mero de serie del volumen es: B25A-9483

 Directorio de c:\dev

19/04/2023  20:04    <DIR>          .
12/04/2023  19:21           970.106 bmw.xml
12/04/2023  20:46           674.977 bmw_mazda.csv
08/02/2023  20:32    <DIR>          Bot-dino-chrome-master
08/02/2023  20:32             3.520 Bot-dino-chrome-master.zip
24/03/2022  18:42    <DIR>          clonenetflix-main
12/04/2023  20:44           678.108 coches_data.csv
08/04/2023  19:51               742 coches_data.json
12/04/2023  19:44           114.277 coches_link.csv
08/04/2023  19:54            95.799 coches_link2.csv
28/06/2022  21:34    <DIR>          demo
19/04/2023  18:03           319.956 dict_vectorizer_coches_net.pck
19/04/2023  20:08            24.817 Flask_with_Keras.ipynb
24/10/2022  16:30    <DIR>          grumpis
14/02/2023  19:37        15.011.296 Gyazo-4.6.exe
19/04/2023  18:03           986.088 modelo_keras_coches_net.hdf5
19/04/2023  20:04         1.306.334 mode

In [4]:

with open('dict_vectorizer_coches_net.pck', 'rb') as f:
  dv = pickle.load(f)
model = tf.keras.models.load_model('modelo_keras_coches_net.hdf5')
# Crear un nuevo optimizador
optimizer = Adam(learning_rate=0.01)

# Compilar el modelo con el nuevo optimizador
model.compile(loss='mean_squared_error', optimizer=optimizer)

c:\Users\mateo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
model

In [6]:
dv

DictVectorizer(sparse=False)

# Uso del modelo

In [7]:
coche = {
        "color": "GRAY",
        "fueltype": "Gasolina",
        "km": 93000.0,
        "make": "AUDI",
        "model": "Q2",
        "province": "Alicante",
        "title": "AUDI Q2 Design 30 TFSI 85kW 116CV 5p",
        "transmissiontype": "MANUAL",
        "year": 2019.0,
        "seller_type": "profesional",
        "bodytype": "4x4 SUV",
        "cubiccapacity": 999.0,
        "doors": 5.0,
        "hp": 116.0
    }
X_train = dv.transform(coche)
X_train

array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        2.019e+03]])

In [8]:
precio = model.predict(X_train)
precio

1/1 [==============================] - 0s 81ms/step


array([[18484.176]], dtype=float32)

In [12]:
from flask import Flask, request, Response

#pip install -U flask-cors
from flask_cors import CORS

application = Flask(__name__)
CORS(application)

with open('dict_vectorizer_coches_net.pck', 'rb') as f:
  dv = pickle.load(f)
model = tf.keras.models.load_model('modelo_keras_coches_net.hdf5')
# Crear un nuevo optimizador
optimizer = Adam(learning_rate=0.01)

# Compilar el modelo con el nuevo optimizador
model.compile(loss='mean_squared_error', optimizer=optimizer)

@application.route('/flask', methods=['GET'])
def flask():
    return '<p>https://flask.palletsprojects.com/en/2.2.x/</p>'

@application.get('/api/get')
def get_method():
    word = request.args.get('word', '<no word>')
    return {
        'hello': 'hello, ' + word
    }
    
@application.post('/api/predict/coche')
def predict_car():
    coche = request.get_json()
    print(coche)
    if coche is not None:
        print('data:',[dict(coche)])
        X_train = dv.transform([dict(coche)])
        precio = model.predict(X_train)
        print('precio', precio)
        return {'precio': round(float(precio[0]), 2)}
    else:
        return 'No valido!'

@application.route('/')
def main():
    return '<p>Hello, World!</p>'

application.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2023 19:23:00] "OPTIONS /api/predict/coche HTTP/1.1" 200 -


{'color': 'BLUE', 'fueltype': 'Diésel', 'km': '36900', 'make': 'AUDI', 'model': 'A3', 'province': 'Almería', 'transmissiontype': 'AUTOMATIC', 'year': '2017', 'seller_type': 'profesional', 'bodytype': 'Berlina', 'cubiccapacity': '1598.0', 'doors': '5', 'hp': '119'}
data: [{'color': 'BLUE', 'fueltype': 'Diésel', 'km': '36900', 'make': 'AUDI', 'model': 'A3', 'province': 'Almería', 'transmissiontype': 'AUTOMATIC', 'year': '2017', 'seller_type': 'profesional', 'bodytype': 'Berlina', 'cubiccapacity': '1598.0', 'doors': '5', 'hp': '119'}]
1/1 [==============================] - 0s 48ms/step


127.0.0.1 - - [20/Apr/2023 19:23:00] "POST /api/predict/coche HTTP/1.1" 200 -


precio [[19826.854]]


...

precio [[19826.854]]


127.0.0.1 - - [20/Apr/2023 19:30:34] "OPTIONS /api/predict/coche HTTP/1.1" 200 -


{'color': 'BLUE', 'fueltype': 'Diésel', 'km': '38000', 'make': 'AUDI', 'model': 'A7', 'province': 'Almería', 'transmissiontype': 'AUTOMATIC', 'year': '2017', 'seller_type': 'profesional', 'bodytype': 'Berlina', 'cubiccapacity': '3000', 'doors': '5', 'hp': '250'}
data: [{'color': 'BLUE', 'fueltype': 'Diésel', 'km': '38000', 'make': 'AUDI', 'model': 'A7', 'province': 'Almería', 'transmissiontype': 'AUTOMATIC', 'year': '2017', 'seller_type': 'profesional', 'bodytype': 'Berlina', 'cubiccapacity': '3000', 'doors': '5', 'hp': '250'}]
1/1 [==============================] - 0s 31ms/step


127.0.0.1 - - [20/Apr/2023 19:30:34] "POST /api/predict/coche HTTP/1.1" 200 -


precio [[19826.854]]
